In [ ]:
!pip install timm ellzaf_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 37.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.w

In [ ]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torchvision.transforms import v2
from torch.utils.data import DataLoader, ConcatDataset
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import numpy as np
import time
from sklearn.metrics import f1_score
import timm
from timm.models.layers import trunc_normal_
from ellzaf_ml.tools import EarlyStopping

In [ ]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Detectedface.zip -d /content

Streaming output truncated to the last 5000 lines.
  inflating: /content/Detectedface/ImposterFace/0005/0005_00_01_02_134.jpg  
  inflating: /content/Detectedface/ImposterFace/0005/0005_00_01_02_152.jpg  
  inflating: /content/Detectedface/ImposterFace/0005/0005_00_01_02_170.jpg  
  inflating: /content/Detectedface/ImposterFace/0005/0005_00_01_02_189.jpg  
  inflating: /content/Detectedface/ImposterFace/0005/0005_00_01_02_206.jpg  
  inflating: /content/Detectedface/ImposterFace/0005/0005_00_01_02_224.jpg  
  inflating: /content/Detectedface/ImposterFace/0005/0005_00_01_02_246.jpg  
  inflating: /content/Detectedface/ImposterFace/0005/0005_00_01_02_264.jpg  
  inflating: /content/Detectedface/ImposterFace/0005/0005_00_01_02_283.jpg  
  inflating: /content/Detectedface/ImposterFace/0005/0005_00_01_02_300.jpg  
  inflating: /content/Detectedface/ImposterFace/0005/0005_00_01_02_319.jpg  
  inflating: /content/Detectedface/ImposterFace/0005/0005_00_01_02_337.jpg  
  inflating: /content/Det

In [ ]:
%%bash

DETECTEDFACE_PATH="/content/Detectedface"
CLIENTFACE_PATH="$DETECTEDFACE_PATH/ClientFace"
IMPOSTERFACE_PATH="$DETECTEDFACE_PATH/ImposterFace"
FACE_TRAIN_PATH="face_train"
FACE_VAL_PATH="face_val"
FACE_TEST_PATH="face_test"

rm -rf "$FACE_TRAIN_PATH"
rm -rf "$FACE_VAL_PATH"
rm -rf "$FACE_TEST_PATH"

# Rename ImposterFace folders to ensure consistent numbering
# Number 13 was skipped
for i in {14..16}; do
  old_folder="$IMPOSTERFACE_PATH/$(printf '00%02d' $i)"
  new_folder="$IMPOSTERFACE_PATH/$(printf '00%02d' $((i-1)))"
  if [ -d "$old_folder" ]; then
    mv "$old_folder" "$new_folder"
  fi
done

mkdir -p "$FACE_TRAIN_PATH/fake"
mkdir -p "$FACE_TRAIN_PATH/real"
mkdir -p "$FACE_VAL_PATH/fake"
mkdir -p "$FACE_VAL_PATH/real"
mkdir -p "$FACE_TEST_PATH/fake"
mkdir -p "$FACE_TEST_PATH/real"

copy_folders() {
  local src_base=$1
  local dst_base=$2
  shift 2
  local folder_numbers=("$@")

  for num in "${folder_numbers[@]}"; do
    src_folder="$src_base/$(printf '00%02d' $num)"
    dst_folder="$dst_base/$(printf '00%02d' $num)"
    if [ -d "$src_folder" ]; then
      cp -r "$src_folder" "$dst_folder"
    fi
  done
}

odd_numbers=(1 3 5 7 9 11 13 15)
copy_folders "$CLIENTFACE_PATH" "$FACE_TRAIN_PATH/real" "${odd_numbers[@]}"
copy_folders "$IMPOSTERFACE_PATH" "$FACE_TRAIN_PATH/fake" "${odd_numbers[@]}"

val_numbers=(2 4 6)
copy_folders "$CLIENTFACE_PATH" "$FACE_VAL_PATH/real" "${val_numbers[@]}"
copy_folders "$IMPOSTERFACE_PATH" "$FACE_VAL_PATH/fake" "${val_numbers[@]}"

test_numbers=(8 10 12 14)
copy_folders "$CLIENTFACE_PATH" "$FACE_TEST_PATH/real" "${test_numbers[@]}"
copy_folders "$IMPOSTERFACE_PATH" "$FACE_TEST_PATH/fake" "${test_numbers[@]}"

echo "Dataset organized successfully!"

Dataset organized successfully!


In [ ]:
EPOCHS = 200
PATIENCE = 3
BATCH_SIZE = 128
NUM_WORKERS = 12
EPOCH_LEN = len(str(EPOCHS))
BASE_FOLDER = "/content/Detectedface/"

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

device = torch.device(device)

Using cuda device


In [ ]:
data_transforms = {
  "original": v2.Compose([
    v2.Resize(224),
    v2.CenterCrop(224),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ]),

  "flipped": v2.Compose([
    v2.Resize(224),
    v2.CenterCrop(224),
    v2.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    v2.RandomHorizontalFlip(p=1),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ]),

  "spoof": v2.Compose([
    v2.Resize(224),
    v2.CenterCrop(224),
    v2.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
    v2.RandomHorizontalFlip(p=1),
    v2.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    v2.RandomGrayscale(p=0.1),
    v2.ToTensor(),
  ])
}

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [ ]:
class CustomDataset(datasets.ImageFolder):
  def __init__(self, root, special_augment_transform=None, general_augment_transform=None, special_classes=None):
    super().__init__(root)

    if special_augment_transform is not None and not callable(special_augment_transform):
        raise ValueError("special_augment_transform must be a callable or None")
    if general_augment_transform is not None and not callable(general_augment_transform):
        raise ValueError("general_augment_transform must be a callable or None")

    if special_classes is not None:
        if not isinstance(special_classes, (set, list, tuple)):
            raise TypeError("special_classes must be a set, list, or tuple")
        self.special_classes = set(special_classes)
    else:
        self.special_classes = set()

    self.special_augment_transform = special_augment_transform
    self.general_augment_transform = general_augment_transform

  def __getitem__(self, index):
    image, label = super().__getitem__(index)
    class_name = self.classes[label]
    if class_name in self.special_classes and self.special_augment_transform:
      image = self.special_augment_transform(image)
    elif self.general_augment_transform:
      image = self.general_augment_transform(image)

    return image, label

In [ ]:
train_orig = datasets.ImageFolder("face_train", transform=data_transforms["original"])
train_flip = CustomDataset(
  root="face_train",
  general_augment_transform=data_transforms["flipped"],
  special_augment_transform=data_transforms["spoof"],
  special_classes=['fake']
)

train_data_combined = ConcatDataset([train_orig, train_flip])
train_loader = DataLoader(train_data_combined, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)

val_orig = datasets.ImageFolder("face_val", transform=data_transforms["original"])
val_flip = CustomDataset(
  root="face_val",
  general_augment_transform=data_transforms["flipped"],
  special_augment_transform=data_transforms["spoof"],
  special_classes=['fake'],
)

val_data_combined = ConcatDataset([val_orig, val_flip])
val_loader = DataLoader(val_data_combined, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)

test_orig = datasets.ImageFolder("face_test", transform=data_transforms["original"])
test_loader = DataLoader(test_orig, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)

In [ ]:
print(len(train_data_combined),len(val_data_combined),len(test_orig))

13510 6286 2716


In [ ]:
def train_one_epoch(model, train_loader, device, optimizer, criterion, scheduler_lr=None):
  model.train()
  total_loss, total_f1, total_correct, total_samples, total_grad = 0, 0, 0, 0, 0

  for idx, (train_x_data, train_y_data) in enumerate(train_loader):
    train_x_data, train_y_data = train_x_data.to(device), train_y_data.to(device)
    optimizer.zero_grad()
    y_pred = model(train_x_data)

    loss = criterion(y_pred, train_y_data)
    loss.backward()
    grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)
    optimizer.step()

    _, y_pred_max = torch.max(y_pred, dim=1)
    f1_score_val = f1_score(train_y_data.cpu(), y_pred_max.cpu(), average="macro")
    total_correct += (y_pred_max == train_y_data).sum().item()
    total_samples += train_y_data.size(0)
    total_loss += loss.item()
    total_f1 += f1_score_val.item()
    total_grad += grad_norm

    if idx % 10 == 0 and idx > 0:
      lr = optimizer.param_groups[0]['lr']
      print(
        f'[{e:>{EPOCH_LEN}}/{EPOCHS:>{EPOCH_LEN}}] [{idx}/{len(train_loader)}] '
        f'lr {lr:.10f} | loss {loss:.10f} ({total_loss/idx:.4f}) | '
        f'grad_norm {grad_norm:.4f} ({total_grad/idx:.4f}) | '
      )

  if scheduler_lr:
    scheduler_lr.step()
  avg_loss = total_loss / len(train_loader)
  avg_f1 = total_f1 / len(train_loader)
  avg_grad = total_grad / len(train_loader)
  accuracy = total_correct / total_samples

  return avg_loss, avg_f1, accuracy, avg_grad

In [ ]:
def validate_one_epoch(model, val_loader, device, criterion, scheduler_lr=None):
  model.eval()
  total_loss, total_f1, total_correct, total_samples = 0, 0, 0, 0

  with torch.no_grad():
    for val_x_data, val_y_data in val_loader:
      val_x_data, val_y_data = val_x_data.to(device), val_y_data.to(device)
      y_val_pred = model(val_x_data)
      val_loss = criterion(y_val_pred, val_y_data)

      _, y_val_pred_max = torch.max(y_val_pred, dim=1)
      f1_score_val = f1_score(val_y_data.cpu(), y_val_pred_max.cpu(), average="macro")
      total_correct += (y_val_pred_max == val_y_data).sum().item()
      total_samples += val_y_data.size(0)
      total_loss += val_loss.item()
      total_f1 += f1_score_val.item()

  avg_loss = total_loss / len(val_loader)
  if scheduler_lr:
    scheduler_lr.step(avg_loss)

  avg_f1 = total_f1 / len(val_loader)
  accuracy = total_correct / total_samples
  return avg_loss, avg_f1, accuracy

In [ ]:
model = timm.create_model('vit_base_patch16_224.augreg_in21k_ft_in1k', pretrained=True)
model.head = torch.nn.Linear(model.head.in_features, 2)
trunc_normal_(model.head.weight, mean=0.0, std=0.02)
model = model.to(device)

criterion = nn.CrossEntropyLoss()

lr = 3e-5
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler_linear = lr_scheduler.LinearLR(optimizer, start_factor=0.01, total_iters=10)
scheduler_cosine = lr_scheduler.CosineAnnealingLR(optimizer, T_max=490, eta_min=lr/100)
scheduler_lr = lr_scheduler.SequentialLR(optimizer, [scheduler_linear,scheduler_cosine],milestones=[10])

early_stopping = EarlyStopping(
  patience=PATIENCE,
  verbose=True,
  path=f"/content/drive/MyDrive/COS30082/vit_teacher.pth",
)

start_time = time.time()
for e in range(1, EPOCHS + 1):
  avg_loss_train, avg_f1_train, avg_accuracy_train, avg_grad = train_one_epoch(
    model, train_loader, device,
    optimizer, criterion, scheduler_lr,
  )
  avg_loss_val, avg_f1_val, avg_accuracy_val = validate_one_epoch(model, val_loader, device, criterion)

  time_taken = time.time() - start_time
  time_format = time.strftime("%H:%M:%S", time.gmtime(time_taken))

  print(
    f"[{e:>{EPOCH_LEN}}/{EPOCHS:>{EPOCH_LEN}}] Loss: {avg_loss_train:.5f} | "
    + f"F1-score: {avg_f1_train:.3f} | Acc: {avg_accuracy_train:.3f} | "
    + f"Val Loss: {avg_loss_val:.3f} | Val F1: {avg_f1_val:.3f} | "
    + f"Val Acc: {avg_accuracy_val:.3f} | {time_format}s | "
    + f"Grad: {avg_grad:.5f}"
  )


  early_stopping(avg_loss_val, model)
  if early_stopping.early_stop:
    print(f"Early stopping after {e} Epochs")
    break

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

[  1/200] [10/106] lr 0.0000003000 | loss 0.5723170042 (0.7768) | grad_norm 25.4500 (32.9659) | 
[  1/200] [20/106] lr 0.0000003000 | loss 0.4664854407 (0.6616) | grad_norm 20.8990 (27.6274) | 
[  1/200] [30/106] lr 0.0000003000 | loss 0.3203960955 (0.5696) | grad_norm 13.3299 (23.9998) | 
[  1/200] [40/106] lr 0.0000003000 | loss 0.2501043677 (0.4972) | grad_norm 12.5493 (21.3733) | 
[  1/200] [50/106] lr 0.0000003000 | loss 0.1839806736 (0.4437) | grad_norm 8.4728 (19.2573) | 
[  1/200] [60/106] lr 0.0000003000 | loss 0.1400035620 (0.3973) | grad_norm 6.9508 (17.5506) | 
[  1/200] [70/106] lr 0.0000003000 | loss 0.0899394155 (0.3580) | grad_norm 4.6018 (16.0021) | 
[  1/200] [80/106] lr 0.0000003000 | loss 0.0564098768 (0.3241) | grad_norm 3.9676 (14.6294) | 
[  1/200] [90/106] lr 0.0000003000 | loss 0.0634391904 (0.2955) | grad_norm 3.7216 (13.5417) | 
[  1/200] [100/106] lr 0.0000003000 | loss 0.0382347703 (0.2706) | grad_norm 2.5512 (12.5118) | 
[  1/200] Loss: 0.25750 | F1-score:

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/COS30082/spoof.pth")

In [ ]:
model = timm.create_model('vit_base_patch16_224.augreg_in21k_ft_in1k', pretrained=True)
model.head = torch.nn.Linear(model.head.in_features, 2)
model = model.to(device)
model.load_state_dict(
  torch.load(
    "/content/drive/MyDrive/COS30082/vit_teacher.pth"
  )
)
model.eval()

with torch.no_grad():
  correct = 0
  tp = 0
  tn = 0
  fp = 0
  fn = 0

  for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    output = model(data)
    pred = output.argmax(dim=1, keepdim=True)

    tp += (pred.eq(1) & target.eq(1).view_as(pred)).sum().item()
    tn += (pred.eq(0) & target.eq(0).view_as(pred)).sum().item()
    fp += (pred.eq(1) & target.eq(0).view_as(pred)).sum().item()
    fn += (pred.eq(0) & target.eq(1).view_as(pred)).sum().item()

    correct += pred.eq(target.view_as(pred)).sum().item()

  accuracy = correct / len(test_loader.dataset)
  far = fp / (fp + tn)
  frr = fn / (fn + tp)

  recall = tp / (tp + fn)

  hter = (far + frr ) / 2

  print(f"test acc: {accuracy * 100}%")
  print(f"recall: {recall * 100}%")
  print(f"far: {far * 100}%")
  print(f"frr: {frr * 100}%")
  print(f"hter: {hter * 100}%")

test acc: 99.77908689248895%
recall: 99.45994599459947%
far: 0.0%
frr: 0.54005400540054%
hter: 0.27002700270027%
